In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from test_tool import multiply, load_cv_info
from CV import tailor_CV

# Create the agent
memory = MemorySaver()
model = init_chat_model("openai:gpt-4")
tools = [multiply, load_cv_info, tailor_CV]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Generate a tailored CV.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Generate a tailored CV.
================================== Ai Message ==================================
Tool Calls:
  tailor_CV (call_X2t9fuyYDg6cXKOlfDD4zJ4A)
 Call ID: call_X2t9fuyYDg6cXKOlfDD4zJ4A
  Args:
================================= Tool Message =================================
Name: tailor_CV

Fergal Riordan  
Ireland (Willing to Relocate) | +353 838754381 | fergalriordan333@gmail.com | Portfolio Website | GitHub | LinkedIn  

PROFILE  
Software Engineer with a First-Class Honours MEng in Electronic & Computer Engineering from Trinity College Dublin. Experience spans Agentic AI development for a Partner Relationship Management (PRM) platform, RLHF data annotation for LLMs, and generative AI research in CycleGAN architectures. Proficient in Python and PyTorch, with a passion for building responsible, efficient AI solutions.  

TECHNICAL SKILLS  
Key Competencies: Agentic AI, Generative AI, Prom

In [ ]:
from openai import OpenAI
client = OpenAI()

with open("job_description.txt") as f:
    job_listing = f.read()

response = client.responses.create(
   input=f"""
    {job_listing}

    Return the tailored CV as a JSON object with fields:
    Profile, Technical Skills, Relevant Projects
    """,
   prompt={
       "id": "pmpt_68b0c358291c81968a00e9414e386276009a458f13092457",
       "version": "13"
   },
   text={
       "format": {
           "type": "json_schema",
           "name": "cv_schema",
           "schema": {
               "type": "object",
               "properties": {
                   "Profile": {
                       "type": "string",
                       "description": "A tailored CV profile section of approximately 50 words."
                   },
                   "Technical Skills": {
                       "type": "object",
                       "properties": {
                           "Key Competencies": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 3,
                               "maxItems": 3
                           },
                           "Programming Languages": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 4,
                               "maxItems": 6
                           },
                           "Frameworks & Libraries": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 4,
                               "maxItems": 6
                           },
                           "Tools & Platforms": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 4,
                               "maxItems": 6
                           }
                       },
                       "required": [
                           "Key Competencies",
                           "Programming Languages",
                           "Frameworks & Libraries",
                           "Tools & Platforms"
                       ],
                       "additionalProperties": False
                   },
                   "Relevant Projects": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "Title": {"type": "string"},
                                "Skills": {"type": "string"}
                            },
                            "required": ["Title", "Skills"],
                            "additionalProperties": False
                        },
                        "minItems": 2,
                        "maxItems": 2
                    }
               },
               "required": ["Profile", "Technical Skills", "Relevant Projects"],
               "additionalProperties": False
           },
           "strict": True
       }
   },
)


In [13]:
import json

cv_data = json.loads(response.output_text)
print(json.dumps(cv_data, indent=2))

{
  "Profile": "Software Engineer with a strong foundation in Python and AI agent development, demonstrated through hands-on experience in building scalable AI solutions for PRM platforms. Proficient in LLMs and generative AI, with a focus on creating efficient, high-impact software. Eager to contribute to a dynamic team and grow within a fast-paced environment.",
  "Technical Skills": {
    "Key Competencies": [
      "Agentic AI",
      "Generative AI",
      "AI Research"
    ],
    "Programming Languages": [
      "Python",
      "C",
      "SQL",
      "C++"
    ],
    "Frameworks & Libraries": [
      "LangChain",
      "PyTorch",
      "OpenAI API",
      "NumPy"
    ],
    "Tools & Platforms": [
      "Docker",
      "Azure",
      "Cosmos DB",
      "Git"
    ]
  },
  "Relevant Projects": [
    {
      "Title": "Master's Thesis: Enhancing CycleGAN for Day-to-Night Image Translation",
      "Skills": "Python, PyTorch, GANs, Transfer Learning, Deep Learning"
    },
    {
      "

In [67]:
with open("projects.txt", "r", encoding="utf-8") as f:
    projects = json.load(f)

for i, project in enumerate(cv_data["Relevant Projects"]):
    project_title = project["Title"]
    if project_title in projects:
        details = projects[project_title]
        cv_data["Relevant Projects"][i]["Dates"] = details.get("Dates", "")
        cv_data["Relevant Projects"][i]["Link"] = details.get("Link", "")
        cv_data["Relevant Projects"][i]["Description"] = details.get("Description", "")


In [68]:
print(cv_data)

{'Profile': 'Software Engineer with a strong foundation in Python and AI agent development, demonstrated through hands-on experience in building scalable AI solutions for PRM platforms. Proficient in LLMs and generative AI, with a focus on creating efficient, high-impact software. Eager to contribute to a dynamic team and grow within a fast-paced environment.', 'Technical Skills': {'Key Competencies': ['Agentic AI', 'Generative AI', 'AI Research'], 'Programming Languages': ['Python', 'C', 'SQL', 'C++'], 'Frameworks & Libraries': ['LangChain', 'PyTorch', 'OpenAI API', 'NumPy'], 'Tools & Platforms': ['Docker', 'Azure', 'Cosmos DB', 'Git']}, 'Relevant Projects': [{'Title': "Master's Thesis: Enhancing CycleGAN for Day-to-Night Image Translation", 'Skills': 'Python, PyTorch, GANs, Transfer Learning, Deep Learning', 'Dates': 'Sep 2023 — Apr 2024', 'Link': 'https://fergalriordan.github.io/projects/cyclegan', 'Description': ['Improved CycleGAN performance using transfer learning, a content-sty

In [ ]:
from docxtpl import DocxTemplate, RichText

def render_cv_to_docx(cv_data: dict, template_file="cv_template.docx", output_file="tailored_cv.docx"):
    doc = DocxTemplate(template_file)

    # Build RichText hyperlinks for the two projects
    project_1_link = RichText()
    project_1_link.add("View Project", url_id=doc.build_url_id(cv_data["Relevant Projects"][0]["Link"]))

    project_2_link = RichText()
    project_2_link.add("View Project", url_id=doc.build_url_id(cv_data["Relevant Projects"][1]["Link"]))

    # Rename keys for Jinja (no spaces, consistent with template placeholders)
    context = {
        "Profile": cv_data["Profile"],
        "Key_Competencies": cv_data["Technical Skills"]["Key Competencies"],
        "Programming_Languages": cv_data["Technical Skills"]["Programming Languages"],
        "Frameworks_and_Libraries": cv_data["Technical Skills"]["Frameworks & Libraries"],
        "Tools_and_Platforms": cv_data["Technical Skills"]["Tools & Platforms"],
        "Project_1_Title": cv_data["Relevant Projects"][0]["Title"],
        "Project_1_Dates": cv_data["Relevant Projects"][0]["Dates"],
        "Project_1_Skills": cv_data["Relevant Projects"][0]["Skills"],
        "Project_1_Link": project_1_link,
        "Project_1_Description": cv_data["Relevant Projects"][0]["Description"],
        "Project_2_Title": cv_data["Relevant Projects"][1]["Title"],
        "Project_2_Dates": cv_data["Relevant Projects"][1]["Dates"],
        "Project_2_Skills": cv_data["Relevant Projects"][1]["Skills"],
        "Project_2_Link": project_2_link,
        "Project_2_Description": cv_data["Relevant Projects"][1]["Description"],
    }

    doc.render(context)
    doc.save(output_file)
    print(f"CV saved to {output_file}")


In [80]:
render_cv_to_docx(cv_data=cv_data)

CV saved to tailored_cv.docx


In [1]:
from docx2pdf import convert

convert("tailored_cv.docx", "tailored_cv.pdf")

  0%|          | 0/1 [00:00<?, ?it/s]